In [1]:
import numpy as np

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs


In [3]:
import matplotlib.pyplot as plt

def plot(paths_XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colours = ['red', 'green', 'blue', 'orange', 'purple', 'cyan']
    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()


In [2]:
from sklearn.linear_model import LinearRegression
import numpy as np

def regularize_curve(XY):
    model = LinearRegression().fit(XY[:, 0].reshape(-1, 1), XY[:, 1])
    if np.allclose(XY[:, 1], model.predict(XY[:, 0].reshape(-1, 1))):
        return "Line"
   
    center = np.mean(XY, axis=0)
    distances = np.linalg.norm(XY - center, axis=1)
    if np.allclose(distances, np.mean(distances)):
        return "Circle"
   
    if len(XY) == 3:
        return "Triangle"
   
    def is_rectangle(XY):
        if len(XY) == 4:
            side_lengths = np.linalg.norm(np.diff(XY, axis=0, append=XY[:1]), axis=1)
            diagonals = np.linalg.norm(XY[0] - XY[2]), np.linalg.norm(XY[1] - XY[3])
            return np.allclose(side_lengths[0], side_lengths[2]) and np.allclose(side_lengths[1], side_lengths[3]) and np.allclose(diagonals[0], diagonals[1])
        return False
    
    def is_square(XY):
        if is_rectangle(XY):
            side_lengths = np.linalg.norm(np.diff(XY, axis=0, append=XY[:1]), axis=1)
            return np.allclose(side_lengths[0], side_lengths)
        return False
    
    if is_square(XY):
        return "Square"
    
    if is_rectangle(XY):
        return "Rectangle"
    def is_rhombus(XY):
        if len(XY) == 4:
            side_lengths = np.linalg.norm(np.diff(XY, axis=0, append=XY[:1]), axis=1)
            return np.allclose(side_lengths, np.mean(side_lengths))
        return False
    
    if is_rhombus(XY):
        return "Rhombus"
    
    return "Unknown"

def regularize_paths(paths_XYs):
    results = []
    for path in paths_XYs:
        result = []
        for XY in path:
            shape_type = regularize_curve(XY)
            result.append((XY, shape_type))
        results.append(result)
    return results


In [4]:
# Symmetry Detection
def detect_symmetry(XY):
    midpoint = np.mean(XY, axis=0)
    reflected = 2 * midpoint - XY
    if np.allclose(XY, reflected):
        return "Symmetric"
    return "Not Symmetric"

def check_symmetry(paths_XYs):
    results = []
    for path in paths_XYs:
        result = []
        for XY in path:
            symmetry = detect_symmetry(XY)
            result.append((XY, symmetry))
        results.append(result)
    return results


In [8]:
import numpy as np

def fit_circle(XY):
    A = np.hstack([2 * XY, np.ones((XY.shape[0], 1))])
    b = np.sum(XY**2, axis=1)
    c = np.linalg.lstsq(A, b, rcond=None)[0]
    center = c[:2]
    radius = np.sqrt(c[2] + np.sum(center**2))
    return center, radius

def complete_curve(XY, shape_type="Unknown"):
    if shape_type == "Line":
        if len(XY) > 2:
            return np.vstack([XY, 2 * XY[-1] - XY[-2]])
    
    elif shape_type == "Circle":
        center, radius = fit_circle(XY)
        angle_start = np.arctan2(XY[0, 1] - center[1], XY[0, 0] - center[0])
        angle_end = np.arctan2(XY[-1, 1] - center[1], XY[-1, 0] - center[0])
        arc_angles = np.linspace(angle_start, angle_end, num=100)
        arc_points = np.array([center + radius * np.array([np.cos(a), np.sin(a)]) for a in arc_angles])
        return np.vstack([XY, arc_points])
    
    elif shape_type == "Arc":
        center, radius = fit_circle(XY)
        arc_angle = np.arctan2(XY[-1, 1] - center[1], XY[-1, 0] - center[0]) - np.arctan2(XY[0, 1] - center[1], XY[0, 0] - center[0])
        if abs(arc_angle) < np.pi:
            extended_angle = np.linspace(0, 2 * np.pi, num=100)
            extended_arc = np.array([center + radius * np.array([np.cos(a), np.sin(a)]) for a in extended_angle])
            return np.vstack([XY, extended_arc])
    return XY

def complete_paths(paths_XYs, shape_types):
    results = []
    for path, shape_type in zip(paths_XYs, shape_types):
        result = []
        for XY in path:
            completed = complete_curve(XY, shape_type)
            result.append(completed)
        results.append(result)
    return results
paths_XYs = [
    np.array([[0, 0], [1, 1], [2, 2]]),  
    np.array([[1, 1], [2, 0], [3, 1]]), 
]
shape_types = ["Line", "Arc"]
completed_results = complete_paths(paths_XYs, shape_types)

for i, path in enumerate(completed_results):
    print(f"Completed Curve {i+1}:")
    for XY in path:
        print(f"  Points: {XY}")


In [11]:
import svgwrite
import cairosvg

def polylines2svg(paths_XYs, svg_path):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)
    
    dwg = svgwrite.Drawing(svg_path, profile='tiny', shape_rendering='crispEdges')
    group = dwg.g()
    colours = ['red', 'green', 'blue', 'orange', 'purple', 'cyan']
    for i, path in enumerate(paths_XYs):
        path_data = []
        c = colours[i % len(colours)]
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            if not np.allclose(XY[0], XY[-1]):
                path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill=c, stroke='none', stroke_width=2))
    dwg.add(group)
    dwg.save()
    
    png_path = svg_path.replace('.svg', '.png')
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(url=svg_path, write_to=png_path, parent_width=W, parent_height=H, output_width=fact * W, output_height=fact * H, background_color='white')


In [5]:

input_csv = 'examples/isolated.csv'  
output_svg = 'output/isolated.svg'
    
paths_XYs = read_csv(input_csv)
plot(paths_XYs)
regularized = regularize_paths(paths_XYs)
symmetry = check_symmetry(paths_XYs)
completed = complete_paths(paths_XYs)
polylines2svg(completed, output_svg)